In [1]:
1+1

2

In [3]:
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

     ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
     ---------------------------------------- 68.0/68.0 kB 3.8 MB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB ? eta 0:00:00
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with stat


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
from glob import glob
from pprint import pprint
import os

In [18]:
load_dotenv()

True

In [6]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
    length_function =len,
    separators= ['\n\n', '\n']
)

loader = Docx2txtLoader('./data/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)
print(len(document_list))
print(f'각 청크 길이 : {list(len(text.page_content) for text in document_list)}')



225
각 청크 길이 : [1466, 1421, 1427, 1396, 1398, 1481, 1251, 1483, 1471, 1480, 1479, 1425, 1494, 1458, 1492, 1482, 1493, 1429, 1363, 1437, 1463, 1450, 1481, 1437, 1441, 1456, 1433, 1319, 1458, 1496, 1461, 1382, 1384, 1479, 1494, 1435, 1376, 1479, 1470, 1481, 1333, 1412, 1477, 1014, 1456, 1362, 1446, 1386, 1055, 1467, 1361, 1493, 1473, 1493, 1428, 1443, 1470, 1486, 1290, 1442, 1421, 1496, 1478, 1288, 1492, 1458, 1441, 1237, 1476, 1416, 1431, 1429, 1445, 1460, 1483, 1399, 1484, 1481, 1333, 1308, 1385, 1479, 1495, 1386, 1375, 1345, 1353, 1382, 1446, 1356, 1409, 1441, 1443, 1431, 1494, 1226, 1413, 1468, 1396, 1455, 1488, 1496, 1446, 1353, 1397, 1427, 1443, 1440, 1478, 1443, 1436, 1458, 1360, 1434, 1429, 1443, 1412, 1494, 1414, 1478, 1471, 1373, 1492, 1429, 1493, 1377, 1438, 1389, 1305, 1446, 1160, 1403, 1465, 1371, 1342, 1427, 1235, 1363, 1460, 1387, 1277, 1473, 1386, 1410, 1460, 1457, 1489, 1488, 1335, 1454, 1411, 1488, 1482, 1448, 1265, 1412, 1352, 1377, 1459, 1451, 1288, 1450, 1481, 1459, 1

In [8]:
# 임베딩 
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    model = 'text-embedding-3-large'
)

In [ ]:
# 벡터DB -> 터진사람이 많아서 크로마디비 버림ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ->로컬디비
# from langchain_chroma import Chroma

# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     collection_name="chroma-tax",
#     persist_directory="./chroma",
# )

In [10]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install -qU pinecone

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# https://app.pinecone.io/ 여기 index에 데이터 뜸.

import os 
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv

load_dotenv()

index_name = 'tax-index'
Pinecone_api_key = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=Pinecone_api_key)
pinecone_index =pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(
    document_list, 
    embedding,
    index_name=index_name
    )

In [105]:
#query = "연봉 5천만원 거주자의 종합소득세는 얼마인가요?"
query = "연봉 5천만원 직장인 소득세는 얼마인가요?"

In [116]:
retriever = vectorstore.as_retriever(
    search_kwargs ={'k':4}
)

retriever.invoke(query)

[Document(id='541a368d-91bb-4547-9230-0852f92a7a53', metadata={'source': './data/tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제

In [117]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model='gpt-4o'
)

In [118]:
# 허깅페이스 모델 가져옴 밑에꺼쓰던지 이거쓰던지 하나만 사용
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# from langchain.prompts import PromptTemplate

# prompt = PromptTemplate(
#     input_variables=['context','question'],
#     template="""
#     [Identity]
#     - 당신은 최고의 한국 소득세 전문가 입니다.
#     - [Context]를 참고해서 간단히 사용자의 질문에 답변해주세요.
#     [Context]
#     {context}

#     Question : {question}
# """
# )


In [119]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs = {"prompt": prompt}
)

In [120]:
ai_message =qa_chain.invoke({"query":query})

In [121]:
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가요?',
 'result': '연봉 5천만원의 소득세는 기본세율에 따라 계산됩니다. 이는 1,400만원 초과 5,000만원 이하에 해당하므로, 84만원에 초과하는 금액의 15%를 더한 금액이 소득세로 부과됩니다. 따라서 대략적인 소득세는 576만원입니다.'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


dictionary =["사람을 나타내는 표현 -> 거주자"]

prompt2 =ChatPromptTemplate.from_template(f'''
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전 : {dictionary}
    질문 : {{question}}
''')

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query":dictionary_chain} |qa_chain


In [ ]:
#테스트용 질문이바뀌는지 안바뀌는지 테스트해봄

new_question = dictionary_chain.invoke({"question": query})
new_question 

'연봉 5천만원 거주자 소득세는 얼마인가요?'

In [124]:
ai_message =tax_chain.invoke({"question":query})

In [125]:
ai_message

{'query': '연봉 5천만원 거주자 소득세는 얼마인가요?',
 'result': '연봉 5천만원 거주자의 소득세는 과세표준 1,400만원 초과 5,000만원 이하 구간에 해당하며, 계산식에 따르면 소득세는 84만원 + (5,000만원 - 1,400만원) * 15% 입니다. 이것은 총 684만원의 소득세가 됩니다.'}